In [ ]:
#!pip install chromadb sentence-transformers openai

In [1]:
import os, sys
import chromadb
from chromadb import PersistentClient
from sentence_transformers import SentenceTransformer
import openai
import ollama
import requests

In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

In [3]:
# Chroma Config
persist_path = "../chroma_db"
collection_name = "class10_english"

In [4]:
# Embedding model
MODEL_NAME = "all-MiniLM-L6-v2"
model = SentenceTransformer(MODEL_NAME)

In [5]:
# LLM Config
USE_OPENAI = False  # Set False to use Ollama
OPENAI_MODEL = "gpt-3.5-turbo"
OLLAMA_MODEL = "llama3.1"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [6]:
# Chroma client (🔁 using PersistentClient)
client = PersistentClient(path=persist_path)
collection = client.get_or_create_collection(collection_name)

In [7]:
def retrieve_chunks(query, top_k=3):
    embedding = model.encode([query])[0].tolist()
    results = collection.query(query_embeddings=[embedding], n_results=top_k)
    return results["documents"][0] if results["documents"] else []


In [8]:
def ask_openai(context, query):
    response = openai.ChatCompletion.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful study tutor."},
            {"role": "user", "content": f"Context:\n{context}\n\nQuestion:\n{query}"}
        ]
    )
    return response['choices'][0]['message']['content']

In [9]:
def ask_ollama(context, query):
    prompt = f"Context:\n{context}\n\nQuestion: {query}\n\nAnswer:"
    response = requests.post("http://localhost:11434/api/generate", json={
        "model": OLLAMA_MODEL,
        "prompt": prompt,
        "system": "You are a teacher and evaluator of class10_english",
        "stream": False
    })
    return response.json()["response"]

In [10]:
def embed_text(text):
    return model.encode([text])[0].tolist()

In [11]:
def stream_chat_response(model_name, chat_history):
    # Stream response from Ollama
    stream = ollama.chat(
        model=model_name,
        messages=chat_history,
        stream=True
    )

    # Collect streamed chunks into full response
    assistant_response = ""
    for chunk in stream:
        chunk_content = chunk["message"]["content"]
        assistant_response += chunk_content
        clear_output(wait=True)
        # display(assistant_response)  # Display cumulative response
        display(HTML(assistant_response.replace("\n", "<br>")))
        sys.stdout.flush()  # Force flush to ensure output is shown
    
    # Append assistant's response to chat history
    chat_history.append({"role": "assistant", "content": assistant_response})
    
    return assistant_response, chat_history

In [ ]:
query = "Teach me one of the poem from the book."
chunks = retrieve_chunks(query)
context = "\n\n---\n\n".join(chunks) if chunks else ""
# Get answer from LLM
answer = ask_openai(context, query) if USE_OPENAI else ask_ollama(context, query)
print("🧠 Q:", query)
print("💬 A:", answer)

In [14]:
# Define context and persona and load the context
chat_history = [{"role": "system", "content": f"You are the best teacher of {collection_name}"}]


In [15]:
query = "Show me the full poem"
chunks = retrieve_chunks(query)
context = "\n\n---\n\n".join(chunks) if chunks else ""
chat_history.append({"role": "user", "content": context + "\n" + query})
assistant_response, chat_history = stream_chat_response(OLLAMA_MODEL, chat_history)


In [16]:
chat_history

[{'role': 'system', 'content': 'You are the best teacher of class10_english'},
 {'role': 'user',
  'content': 'Given below is a poem written by Norman Littleford, which\nmay help you think and compose.\n10 Words and ExprEssions 2 – Class x\nReprint 2025-26\nUnit-1.indd 10 27-Jun-2019 04:21:16 PM\n\n---\n\nbelow.\nand can cheat anyone.\n# give space\n(i) Lencho _______________________ thought God would help . put a full stop\n, insert comma\n^\nhim.\ninsert new word\n(j) Those ____________________________ will succeed.\nSee page nos\n140-141 for more\n(k) God helps those ______________________________\nsuch symbols.\nReprint 2025-26\nUnit-1.indd 7 6/25/2024 12:25:45 PM\n\n---\n\ne\nditinG\n1. Suppose Lencho gets a hundred pesos in his envelope. He\nwrites a letter to God expressing his gratitude. However, he\nhas missed a word in each line. Help him with those words\nso that his letter is complete.\nDear God!\nThank for Thank you for\n(a) sending a hundred pesos help. _______________\n(